# Módulo 2: Preprocesamiento de Datos y Desarrollo Backend
## Clase 7: Uso de SQLAlchemy para Manejar Bases de Datos mediante un ORM 🧱⚙️

### Introducción

En la clase anterior, conectamos nuestra API Flask a una base de datos SQLite utilizando el módulo `sqlite3` y escribiendo SQL directamente. Si bien esto funciona, puede volverse tedioso y propenso a errores a medida que la aplicación crece y las consultas se vuelven más complejas. Además, nos ata a la sintaxis específica de SQLite.

Un **ORM (Object-Relational Mapper)** es una técnica de programación que crea un "puente" entre un lenguaje de programación orientado a objetos (como Python) y una base de datos relacional. Permite interactuar con la base de datos utilizando objetos y métodos de Python en lugar de escribir sentencias SQL directamente.

**SQLAlchemy** es un potente toolkit SQL y ORM para Python. Proporciona un conjunto completo de herramientas para interactuar con bases de datos y es agnóstico respecto al motor de base de datos (soporta PostgreSQL, MySQL, SQLite, Oracle, etc.).

**Flask-SQLAlchemy** es una extensión de Flask que integra SQLAlchemy en tu aplicación Flask, simplificando la configuración y el uso.

**Beneficios de usar un ORM como SQLAlchemy:**
* **Código más Pythónico:** Interactúas con la base de datos usando clases y objetos de Python.
* **Abstracción de SQL:** Reduce la cantidad de SQL que necesitas escribir (especialmente para operaciones CRUD comunes).
* **Portabilidad de Base de Datos:** Facilita el cambio entre diferentes sistemas de bases de datos con cambios mínimos en el código.
* **Seguridad:** Ayuda a prevenir ataques de inyección SQL cuando se usa correctamente.
* **Productividad:** Puede acelerar el desarrollo al manejar tareas repetitivas de base de datos.

### 1. Prerrequisitos y Configuración

Necesitarás Flask y Flask-SQLAlchemy. SQLAlchemy se instalará como una dependencia de Flask-SQLAlchemy.

In [ ]:
# En tu terminal o Anaconda Prompt, ejecuta:
# pip install Flask Flask-SQLAlchemy SQLAlchemy

---

## 2. Configurando Flask con Flask-SQLAlchemy

Primero, configuramos nuestra aplicación Flask para que use SQLAlchemy.
El siguiente bloque de código representa el inicio de un archivo `app_sqlalchemy.py`.

In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import os # Para manejar rutas de archivos de forma robusta

app = Flask(__name__)

# Configuración de la base de datos
# Obtenemos la ruta base del directorio del script actual
# Nota: __file__ no está definido en un notebook, esto es para un script .py
# Para un notebook, podrías definir base_dir de otra manera, ej. base_dir = os.getcwd()
try:
    base_dir = os.path.abspath(os.path.dirname(__file__))
except NameError: # Ocurre si __file__ no está definido (ej. en un notebook interactivo)
    base_dir = os.getcwd() # Usar el directorio de trabajo actual como fallback

# Definimos la URI de la base de datos SQLite. El archivo se creará en base_dir.
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(base_dir, 'tasks_sqlalchemy.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # Desactiva una característica de Flask-SQLAlchemy que no necesitamos y consume recursos

# Inicializar la extensión SQLAlchemy con nuestra aplicación Flask
db = SQLAlchemy(app)

print(f"Base de datos configurada en: {app.config['SQLALCHEMY_DATABASE_URI']}")

**Explicación de la Configuración:**
* `app.config['SQLALCHEMY_DATABASE_URI']`: Le dice a SQLAlchemy dónde encontrar la base de datos. Para SQLite, es una ruta a un archivo.
* `app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False`: Desactiva el sistema de seguimiento de modificaciones de Flask-SQLAlchemy.
* `db = SQLAlchemy(app)`: Crea la instancia de SQLAlchemy, vinculándola a nuestra aplicación Flask.

---

## 3. Definiendo Modelos de Datos con SQLAlchemy

Un modelo es una clase de Python que representa una tabla en tu base de datos. Cada atributo de la clase corresponde a una columna en la tabla.
Continuamos con el contenido del archivo `app_sqlalchemy.py`:

In [ ]:
# --- Modelo --- 
class Task(db.Model): # Hereda de db.Model
    __tablename__ = 'tasks' # Opcional, por defecto usa el nombre de la clase en minúsculas

    id = db.Column(db.Integer, primary_key=True) # Columna ID, entero, clave primaria (autoincremental por defecto)
    titulo = db.Column(db.String(100), nullable=False) # Cadena de hasta 100 chars, no puede ser nulo
    descripcion = db.Column(db.String(200), nullable=True) # Cadena de hasta 200 chars, puede ser nulo
    completada = db.Column(db.Boolean, default=False, nullable=False) # Booleano, por defecto False

    # (Opcional) Representación en string del objeto, útil para debugging
    def __repr__(self):
        return f'<Task {self.id}: {self.titulo}>'

    # Método para convertir el objeto Task a un diccionario (para serialización JSON)
    def to_dict(self):
        return {
            'id': self.id,
            'titulo': self.titulo,
            'descripcion': self.descripcion,
            'completada': self.completada
        }
print("Modelo Task definido.")

**Para crear las tablas en la base de datos (se hace una vez, usualmente al iniciar la app):**
```python
# Esto se ejecutaría en el contexto de la aplicación, por ejemplo, antes de app.run()
# o mediante un comando CLI específico.
# with app.app_context():
#     db.create_all()
# print("Tablas creadas (si no existían).")
```

---

## 4. Operaciones CRUD con SQLAlchemy ORM en la API de Tareas

Ahora modificaremos nuestros endpoints Flask para usar el modelo `Task` y la sesión de SQLAlchemy (`db.session`) para interactuar con la base de datos.
Estos bloques de código también serían parte de `app_sqlalchemy.py`.

#### a) Crear una Tarea (POST /tasks)

In [ ]:
@app.route('/tasks', methods=['POST'])
def create_task():
    data = request.get_json()
    if not data or 'titulo' not in data:
        return jsonify({'error': 'El título es requerido'}), 400

    nueva_tarea = Task(
        titulo=data['titulo'],
        descripcion=data.get('descripcion', ""),
        completada=data.get('completada', False) # Acepta booleano directamente
    )

    try:
        db.session.add(nueva_tarea) # Añade el nuevo objeto a la sesión
        db.session.commit()        # Guarda los cambios en la base de datos
        return jsonify(nueva_tarea.to_dict()), 201
    except Exception as e:
        db.session.rollback() # Revertir en caso de error
        return jsonify({'error': str(e)}), 500

: 

#### b) Obtener Todas las Tareas (GET /tasks)

In [ ]:
@app.route('/tasks', methods=['GET'])
def get_tasks():
    try:
        todas_las_tareas = Task.query.all() # Obtiene todos los registros de la tabla Task
        return jsonify({'tasks': [task.to_dict() for task in todas_las_tareas]})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

#### c) Obtener una Tarea Específica (GET /tasks/<task_id>)

In [ ]:
@app.route('/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    try:
        # .get() es la forma recomendada para buscar por clave primaria
        task = db.session.get(Task, task_id) 
        # task = Task.query.get_or_404(task_id) # Alternativa que lanza 404 si no se encuentra
        if task is None:
           return jsonify({'error': 'Tarea no encontrada'}), 404
        return jsonify(task.to_dict())
    except Exception as e:
        return jsonify({'error': str(e)}), 500

#### d) Actualizar una Tarea (PUT /tasks/<task_id>)

In [ ]:
@app.route('/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    data = request.get_json()
    if not data:
        return jsonify({'error': 'Datos no proporcionados'}), 400

    try:
        task = db.session.get(Task, task_id)
        if task is None:
            return jsonify({'error': 'Tarea no encontrada'}), 404
        
        task.titulo = data.get('titulo', task.titulo)
        task.descripcion = data.get('descripcion', task.descripcion)
        if 'completada' in data:
            if not isinstance(data['completada'], bool):
                return jsonify({'error': 'El campo "completada" debe ser un booleano'}), 400
            task.completada = data['completada']
        
        db.session.commit() # SQLAlchemy rastrea los cambios en 'task' y los guarda
        return jsonify(task.to_dict())
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

#### e) Eliminar una Tarea (DELETE /tasks/<task_id>)

In [ ]:
@app.route('/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    try:
        task = db.session.get(Task, task_id)
        if task is None:
            return jsonify({'error': 'Tarea no encontrada'}), 404
        
        db.session.delete(task) # Marca el objeto para ser eliminado
        db.session.commit()     # Ejecuta la eliminación en la BD
        
        return jsonify({'mensaje': 'Tarea eliminada exitosamente'}), 200
    except Exception as e:
        db.session.rollback()
        return jsonify({'error': str(e)}), 500

### 4.1. Bloque Principal para Ejecutar la Aplicación
Este sería el final de tu archivo `app_sqlalchemy.py`.

In [ ]:
if __name__ == '__main__':
    # Crear las tablas en la base de datos si no existen.
    # Es importante hacerlo dentro del contexto de la aplicación.
    with app.app_context():
        db.create_all()
    app.run(debug=True)

**Nota sobre `db.session.get(Model, id)`:**
A partir de SQLAlchemy 1.4 y Flask-SQLAlchemy 3.0, `db.session.get(Model, id)` es la forma preferida para obtener un objeto por su clave primaria en lugar de `Model.query.get(id)`. Si usas versiones anteriores, `Model.query.get()` sería la forma.

---

## 5. Ejecutando y Probando la API

1.  **Guarda todo el código** (Configuración, Modelo, Endpoints y el bloque `if __name__ == '__main__':`) en un archivo, por ejemplo, `app_sqlalchemy.py`.
2.  **Ejecuta desde la terminal:** `python app_sqlalchemy.py`.
3.  Se creará un archivo `tasks_sqlalchemy.db` (si no existe) y las tablas definidas en tu modelo.
4.  **Prueba los endpoints** usando Postman, `curl` o la librería `requests` de Python. Los datos ahora persistirán entre reinicios del servidor.

---

## 6. Ventajas y Consideraciones de SQLAlchemy ORM

**Ventajas (repaso):**
* **Abstracción del SQL:** Escribes menos SQL.
* **Código Orientado a Objetos:** Más natural en Python.
* **Portabilidad:** Cambiar de SQLite a PostgreSQL, por ejemplo, requiere principalmente cambiar la `SQLALCHEMY_DATABASE_URI`.
* **Manejo de Sesiones y Transacciones:** SQLAlchemy gestiona esto por ti.

**Consideraciones:**
* **Curva de Aprendizaje:** Dominar SQLAlchemy, especialmente sus características avanzadas, lleva tiempo.
* **Rendimiento:** Para consultas extremadamente complejas, el SQL generado por el ORM podría no ser tan eficiente como el SQL escrito a mano, pero para la mayoría de las aplicaciones es adecuado.
* **Migraciones de Esquema:** Para cambios en la estructura de las tablas en producción, se usan herramientas como **Alembic** (con Flask-Migrate).

---

## 7. Resumen

Hemos transformado nuestra API Flask para usar SQLAlchemy ORM, permitiendo interactuar con la base de datos de forma orientada a objetos. Definimos modelos, configuramos Flask-SQLAlchemy y adaptamos nuestras operaciones CRUD.
Este enfoque es muy común y robusto para el desarrollo de aplicaciones Python.

## (Opcional) Ejercicio Práctico 📚

Tomando el ejercicio opcional de la clase anterior donde se te pidió crear una API CRUD para "libros":

1.  **Define un Modelo `Libro` con SQLAlchemy:**
    * Columnas: `id` (Integer, PK), `titulo` (String, not null), `autor` (String, not null), `anio_publicacion` (Integer), `isbn` (String, unique, optional).
    * Añade un método `to_dict()` para serialización.
2.  **Configura Flask-SQLAlchemy** en una nueva aplicación Flask para este modelo de libros (puedes usar una base de datos `libros_sqlalchemy.db`).
3.  **Implementa los Endpoints CRUD para Libros:**
    * `POST /libros`: Crear un nuevo libro.
    * `GET /libros`: Obtener todos los libros.
    * `GET /libros/<int:libro_id>`: Obtener un libro específico por su ID.
    * `PUT /libros/<int:libro_id>`: Actualizar un libro existente.
    * `DELETE /libros/<int:libro_id>`: Eliminar un libro.
4.  Asegúrate de que la tabla `libros` se cree al iniciar la aplicación (`db.create_all()`).
5.  Prueba tu API exhaustivamente.